In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import time

In [2]:
PATH = './parquets'

In [3]:
DB_PARAMS = {
    "dbname": "db",
    "user": "user",
    "password": "pass",
    "host": "localhost",
    "port": 5416
}

In [4]:
INSERT_SQL = """INSERT INTO news (id, titulo, subtitulo, corpo_noticia, url, data_publicacao, embedding) VALUES %s ON CONFLICT (id) DO NOTHING"""

In [5]:
CHUNK_SIZE = 1000

In [6]:
news = pd.read_parquet(f'{PATH}/df_news.parquet')

In [7]:
news.rename(columns={'page': 'id', 'issued_timestamp': 'data_publicacao', 'news_embedding' : 'embedding'}, inplace=True)

In [8]:
news.head()

,id,title,caption,body,url,data_publicacao,embedding
0,13db0ab1-eea2-4603-84c4-f40a876c7400,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,Jeferson da Silva Lima foi escoltado por agent...,"Após audiência de custódia, a Justiça do Amazo...",http://g1.globo.com/am/amazonas/noticia/2022/0...,1655584665000,"[0.06812892061669423, 0.013628176966923193, 0...."
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,Linguajar dos santarenos é diferenciado e chei...,As expressões santarenas não significam apenas...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,http://g1.globo.com/pa/santarem-regiao/noticia...,1561051192000,"[0.3526544359892651, -0.17967424671191085, -0...."
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,Ex-premiê Shinzo Abe morre após ser baleado no...,Ex-primeiro-ministro foi atingido por tiros de...,Novo vídeo mostra que assassino de Shinzo Abe ...,http://g1.globo.com/mundo/noticia/2022/07/08/e...,1657270552000,"[0.0645891399260588, 0.015531811819293855, 0.1..."
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,"Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,"Relator no STF, Fachin vota contra marco tempo...",http://g1.globo.com/politica/noticia/2021/09/0...,1631214406000,"[0.30854146707275, -0.1284120093582454, 0.0683..."
4,9dff71eb-b681-40c7-ac8d-68017ac36675,"\nApós 2 votos, pedido de vista suspende julga...","Pelo marco temporal, índios só podem reivindic...",Após um pedido de vista (mais tempo para análi...,http://g1.globo.com/politica/noticia/2021/09/1...,1631733373000,"[0.5137083142368146, -0.2524442284678125, -0.0..."


In [9]:
news['data_publicacao'] = pd.to_datetime(news['data_publicacao'], unit='ms').dt.tz_localize(None)

In [10]:
news.rename(columns={'title': 'titulo', 'caption' : 'subtitulo', 'body' : 'corpo_noticia'}, inplace=True)

In [11]:
news = news[['id', 'titulo', 'subtitulo','corpo_noticia','url','data_publicacao', 'embedding']]

In [12]:
news.head()

,id,titulo,subtitulo,corpo_noticia,url,data_publicacao,embedding
0,13db0ab1-eea2-4603-84c4-f40a876c7400,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,Jeferson da Silva Lima foi escoltado por agent...,"Após audiência de custódia, a Justiça do Amazo...",http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45,"[0.06812892061669423, 0.013628176966923193, 0...."
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,Linguajar dos santarenos é diferenciado e chei...,As expressões santarenas não significam apenas...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52,"[0.3526544359892651, -0.17967424671191085, -0...."
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,Ex-premiê Shinzo Abe morre após ser baleado no...,Ex-primeiro-ministro foi atingido por tiros de...,Novo vídeo mostra que assassino de Shinzo Abe ...,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52,"[0.0645891399260588, 0.015531811819293855, 0.1..."
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,"Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,"Relator no STF, Fachin vota contra marco tempo...",http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46,"[0.30854146707275, -0.1284120093582454, 0.0683..."
4,9dff71eb-b681-40c7-ac8d-68017ac36675,"\nApós 2 votos, pedido de vista suspende julga...","Pelo marco temporal, índios só podem reivindic...",Após um pedido de vista (mais tempo para análi...,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13,"[0.5137083142368146, -0.2524442284678125, -0.0..."


In [13]:
try:
    conn = psycopg2.connect(**DB_PARAMS)
    cur = conn.cursor()

    for i in range(0, len(news), CHUNK_SIZE):
        chunk = news.iloc[i:i + CHUNK_SIZE]
        values = [(x[0],x[1],x[2],x[3],x[4],x[5],x[6].tolist()) for x in chunk.to_numpy()]
        execute_values(cur, INSERT_SQL, values)
        conn.commit()
        print(f"Chunk {i // CHUNK_SIZE + 1} inserido com sucesso.")

    cur.close()
    conn.close()
    print("Inserção concluída!")

except Exception as e:
    print(f"Erro: {e}")

Chunk 1 inserido com sucesso.
Chunk 2 inserido com sucesso.
Chunk 3 inserido com sucesso.
Chunk 4 inserido com sucesso.
Chunk 5 inserido com sucesso.
Chunk 6 inserido com sucesso.
Chunk 7 inserido com sucesso.
Chunk 8 inserido com sucesso.
Chunk 9 inserido com sucesso.
Chunk 10 inserido com sucesso.
Chunk 11 inserido com sucesso.
Chunk 12 inserido com sucesso.
Chunk 13 inserido com sucesso.
Chunk 14 inserido com sucesso.
Chunk 15 inserido com sucesso.
Chunk 16 inserido com sucesso.
Chunk 17 inserido com sucesso.
Chunk 18 inserido com sucesso.
Chunk 19 inserido com sucesso.
Chunk 20 inserido com sucesso.
Chunk 21 inserido com sucesso.
Chunk 22 inserido com sucesso.
Chunk 23 inserido com sucesso.
Chunk 24 inserido com sucesso.
Chunk 25 inserido com sucesso.
Chunk 26 inserido com sucesso.
Chunk 27 inserido com sucesso.
Chunk 28 inserido com sucesso.
Chunk 29 inserido com sucesso.
Chunk 30 inserido com sucesso.
Chunk 31 inserido com sucesso.
Chunk 32 inserido com sucesso.
Chunk 33 inserido